In [1]:
import os
import pandas as pd

# Compute bleurt and bertscore

In [4]:
from evaluate import load

bleurt = load('bleurt', "BLEURT-20", module_type="metric",
cache_dir='./huggingface_models')
cands = ['hello world']
refs = ['goodbye world']
bleurt.compute(predictions=cands, references=refs)

2023-01-20 18:19:51.571350: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 18:19:53.534291: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-01-20 18:19:53.534382: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-01-20 18:19:53.534389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

INFO:tensorflow:Reading checkpoint ./huggingface_models/bleurt/BLEURT-20/downloads/extracted/0054102ef99aef1b241ed1374740a33a4c8af64cac1c8abc72fa330394a441a2/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: ./huggingface_models/bleurt/BLEURT-20/downloads/extracted/0054102ef99aef1b241ed1374740a33a4c8af64cac1c8abc72fa330394a441a2/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:C

2023-01-20 18:20:03.335624: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 18:20:10.374490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 25029 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0
2023-01-20 18:20:10.377723: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 26037 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-01-20 18:20:10.380717: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:BLEURT initialized.


2023-01-20 18:20:18.377846: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


{'scores': [0.6033896207809448]}

In [5]:
from bert_score import score as score_bert
cands = ['hello world']
refs = ['goodbye world']
def get_bertscore(cands, refs):
    P1, R1, F1 = score_bert(cands, refs, lang="en", 
    model_type='microsoft/deberta-large-mnli',
    batch_size=1, device="cuda:1")
    F1 = F1.cpu().detach().numpy().tolist()
    return F1
F1 = get_bertscore(cands, refs)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaModel: ['config', 'classifier.bias', 'classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# uncomment to save
# for model in ['t5-socnorm',  'dream', 't5-flan', 't5-esnli']:
#     print(model)
#     res = pd.read_csv(f'./data/model_outputs_test_set/{model}.csv')
#     res['bleurt'] = bleurt.compute(predictions=res['expl_pred'], references=res['expl_true'])['scores']
#     res['bertscore'] = get_bertscore(res['expl_pred'].fillna('None').tolist(), 
#                                     res['expl_true'].fillna('None').tolist())
#     res['expl_score'] = (res['bleurt'] + res['bertscore'])/2
#     res.to_csv(f"./data/bert_bleurt_scores/{model}_scored.csv",index=False)

# Compute F1 and explanation score

In [2]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

switch_label = {'Entailment': 'Contradiction',
'No Relation': 'Entailment',
'Contradiction': 'No Relation'}

def adjust_label(y_true, y_pred):
    if y_pred not in {'Entailment', 'Contradiction', 'No Relation'}:
        return switch_label[y_true]
    else:
        return y_pred

def change_label(row, thresh):
    # swithc label to wrong if below thresh
    if row['expl_score'] < thresh:
        return switch_label[row['y_true']]
    else:
        return row['y_pred']

eval_data = []
byclass_data = dict()
for model in ['t5-flan', 'dream', 't5-esnli', 't5-socnorm']:
    data = [model]
    byclass = [model]
    res = pd.read_csv(f"./data/bert_bleurt_scores/{model}_scored.csv")
    res['y_pred'] = res.apply(lambda row: adjust_label(row['y_true'], row['y_pred']), axis=1)
    byclass_data[model] = classification_report(res['y_true'], res['y_pred'], 
                    labels=['Entailment', 'Contradiction', 'No Relation'], zero_division=0, digits=4, output_dict=True)
    f1at0 = round(f1_score(res['y_true'], res['y_pred'], average='macro')*100,2)
    data.append(f1at0)
    for i in [0.5, 0.6, 0.7]:
        res[f'labelAt{i*10}'] = res.apply(lambda row: change_label(row, i), axis=1)
        f1 = round(f1_score(res['y_true'], res[f'labelAt{i*10}'], average='macro')*100,2)
        data.append(f1)
    eval_data.append(data)
eval_df = pd.DataFrame(eval_data, columns=['model', 'F1@0', 'F1@50', 'F1@60', 'F1@70'])
eval_df

,model,F1@0,F1@50,F1@60,F1@70
0,t5-flan,25.85,8.92,7.11,0.76
1,dream,17.68,4.31,0.00,0.00
2,t5-esnli,33.48,8.27,1.14,0.00
3,t5-socnorm,54.52,51.68,43.07,27.45


In [3]:
data = []
for m in byclass_data:
    
    data.append([m,
    round(byclass_data[m]['Entailment']['f1-score']*100,2),
    round(byclass_data[m]['Contradiction']['f1-score']*100,2),
    round(byclass_data[m]['No Relation']['f1-score']*100, 2)])
byclass_df = pd.DataFrame(data, columns=['model', 'Entailment', 
                            'Contradiction', 'NoRelaion'])
byclass_df.to_csv('./data/eval_byclass.csv', index=False)
byclass_df

,model,Entailment,Contradiction,NoRelaion
0,t5-flan,1.36,13.56,62.62
1,dream,27.82,25.21,0.00
2,t5-esnli,21.28,35.63,43.54
3,t5-socnorm,56.60,47.00,59.97
